# NEURAL MACHINE TRANSLATION - Vanilla RNN

## Required Module & Config files

In [1]:
import src.RNN_GRU as rnnNMT
import evaluate
import numpy as np
from torch.nn import CrossEntropyLoss
from torch.optim import NAdam
from torchinfo import summary
from src.Tokenizer import Corpus, LangData, dataLoader
from src.Translator import Translator
from src.utils import load_config, get_device, train_model, sentence_bleu, corpus_bleu

# Loading config file
config = load_config()
# Get device : GPU/MPS Back-End/CPU
device = get_device()
print(f"Using device: {device}")

Using device: mps


## Load the dataset

In [2]:
# Encoder-Source
english_data = Corpus(f"{config.TRAIN_DATA}/english.txt", "English")
afrikaans_data = Corpus(f"{config.TRAIN_DATA}/afrikaans.txt", "Afrikaans")

## Set Hyperparameters

In [3]:
# Encoder - source
IN_ENCODER = english_data.vocab_size
ENCODER_EMB = 256

# Decoder - target
IN_DECODER = afrikaans_data.vocab_size
OUT_DECODER = afrikaans_data.vocab_size
DECODER_EMB = 256

# Shared
HIDDEN_SIZE = 1024
NUM_LAYERS = 2

LR = 1e-3
BATCH_SIZE = 128

## Set the model

In [4]:
encoder_net = rnnNMT.Encoder(IN_ENCODER, ENCODER_EMB, HIDDEN_SIZE, NUM_LAYERS).to(device)
decoder_net = rnnNMT.Decoder(IN_DECODER, DECODER_EMB, HIDDEN_SIZE, NUM_LAYERS).to(device)
model = rnnNMT.RNN_NMT(encoder_net, decoder_net, OUT_DECODER)

summary(model)

Layer (type:depth-idx)                   Param #
RNN_NMT                                  --
├─Encoder: 1-1                           --
│    └─RNN: 2-1                          3,411,968
│    └─Embedding: 2-2                    744,448
├─Decoder: 1-2                           --
│    └─RNN: 2-3                          3,411,968
│    └─Embedding: 2-4                    737,280
│    └─Linear: 2-5                       2,952,000
Total params: 11,257,664
Trainable params: 11,257,664
Non-trainable params: 0

In [5]:
train_data = LangData(english_data, afrikaans_data)
train_loader = dataLoader(train_data, BATCH_SIZE)

pad_idx = afrikaans_data.stoi['<pad>']
criterion = CrossEntropyLoss(ignore_index=0)

optimizer = NAdam(model.parameters(), LR)
translator = Translator(model, english_data, afrikaans_data, device)

In [6]:
# Data used for follow-up durring training
mytext = "<sos> given that we represent the target output as $y\in\{0,1\}$ and we have $n$ training points , we can write the negative log likelihood of the parameters as follows: <eos>"
ground = "<sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as: <eos>"

predicted = translator.translate_sentence(mytext)
bleu = sentence_bleu(prediction=[predicted], reference=[ground])
print(f"Pred: {predicted}")
print(f"Refe: {ground}")
print(f"BLEU SCORES: {bleu}")

Pred: <sos> $\frac{\partial^2j}{\partialw_1^2}>0$ gedagtes bestraf spel bepaal gegaan.""" belaglik hel stilte laar vandag vreemd let wiskundige per gras oggend hardloop tabel aangel\^{e} seuntjie betaal uitree figure geleen lei ideale gedog station\^{e}r moenie (dac) dier ding liefhet dorpie swaar kwaad alhoewel <cld> $x_5=x_1\cdotx_3$ nancy vergelyking tuin skaars elke verjaarsdag \textit{stompies}) murata. lukrake vertaal b duurder maksimum mat antwoordmasjien halloween oorsee papa normalisering engels." tweede-orde kerêl gedoen." (ii) ongemaklik honger line\^{e}r gehoor gebruike romanties getrek $100\textrm{hz}$ straat belaster afskuwelik oorvleueling $\omega=\pi$ $x[n]=x(nt)$ kulture gesteel hele pong vyftig $t=1/6000$ help? "ekskuus gesing verkoue wag storie 17 revier $\sigma_w^2$ ten met geselskap kenmerkvektor) kantoor $\omega=0$ visualisering spesieke verblyf geelkoper teen jorge! parameters geklassifiseer vertaal terwyl intrees $x[0],x[1],\ldots,x[19]$ (kunstenaar) sirkul\^{e}

## Train the data

In [7]:
EPOCHS = 50
params = {
    "model": model,
    "train_loader": train_loader,
    "optimizer": optimizer,
    "criterion": criterion,
    "device": device,
    "epochs": EPOCHS,
    "source_test": mytext,
    "reference": ground,
    "translator": translator
}

train_loss = train_model(**params)
np.save('rnn_train_loss.npy', np.array(train_loss))

Epoch 1/50: 100%|██████████| 20/20 [00:03<00:00,  5.10batch/s, loss=1.793]


Predicted: <sos> (masjienleer) (statistiese $0.25$ deurlaatband $0.35$ $0.35$ banddeurlaatfilter $0.35$ banddeurlaatfilter banddeurlaatfilter banddeurlaatfilter toekomstige $0.35$ $0.35$ toekomstige banddeurlaatfilter banddeurlaatfilter banddeurlaatfilter siens"" data-analitika $0.35$ $0.35$ $0.35$ banddeurlaatfilter banddeurlaatfilter banddeurlaatfilter (bpf) (upsample) toekomstige inferensie) $0.35$ banddeurlaatfilter toekomstige banddeurlaatfilter toekomstige $0.35$ s\^e toekomstige banddeurlaatfilter banddeurlaatfilter (bpf) toekomstige $0.35$ s\^e toekomstige banddeurlaatfilter toekomstige $0.35$ s\^e toekomstige banddeurlaatfilter toekomstige $0.35$ s\^e toekomstige banddeurlaatfilter toekomstige $0.35$ s\^e toekomstige banddeurlaatfilter banddeurlaatfilter (bpf) toekomstige $0.35$ s\^e toekomstige banddeurlaatfilter toekomstige $0.35$ s\^e toekomstige banddeurlaatfilter toekomstige $0.35$ s\^e toekomstige banddeurlaatfilter toekomstige $0.35$ s\^e toekomstige banddeurlaatfilter 

Epoch 2/50: 100%|██████████| 20/20 [00:03<00:00,  5.57batch/s, loss=1.719]


Predicted: <sos> ek is nie <eos>
BLEU Score: [0.024, 0.021, 0.016, 0.0]


Epoch 3/50: 100%|██████████| 20/20 [00:03<00:00,  5.64batch/s, loss=1.469]


...

Predicted: <sos> die sein $x(t)$ bestaan uit 'n sinusgolf met 'n onmiddelike frekwensie wat line\^er verander van $0$ hz to $20$ khz binne 2 sekondes , en dan weer terugdaal na 0 hz in die volgende 2 sekondes <eos>
BLEU Score: [0.315, 0.154, 0.097, 0.0]


Epoch 49/50: 100%|██████████| 20/20 [00:03<00:00,  5.74batch/s, loss=0.565]


Predicted: <sos> die volgende twee eindige-lengte diskrete-tyd seine is onderskeidelik 'n intree wat op die stelsel aangel\^{e} word , $x[n]$ , en die ooreenstemmende uittree , $y[n]$ : <eos>
BLEU Score: [0.426, 0.236, 0.155, 0.096]


Epoch 50/50: 100%|██████████| 20/20 [00:03<00:00,  5.81batch/s, loss=0.585]

Predicted: <sos> die sein $x(t)$ word geanaliseer deur 'n spektrogram te gebruik met die doel om die onmiddelike frekwensie elke 2 ms te bepaal , met 'n frekwensieresolusie van 200 hz en volle hooflob skeiding <eos>
BLEU Score: [0.273, 0.159, 0.107, 0.0]


## Evauate on the Training set

In [8]:
EN_SRC = [' '.join(sent) for sent in english_data.data_str]
AF_REF = [[' '.join(sent)] for sent in afrikaans_data.data_str]
TRANSLATED = [translator.translate_sentence(sent) for sent in EN_SRC]
corpus_bleu(TRANSLATED, AF_REF)

                                     BLEU-1                                     
------------------------------------------------------------------------------------------
bleu                : 0.20163417851385018
precisions          : [0.2016341785138502]
brevity_penalty     : 1.0
length_ratio        : 2.2177799447424023
translation_length  : 81876
reference_length    : 36918
******************************************************************************************
                                     BLEU-2                                     
------------------------------------------------------------------------------------------
bleu                : 0.1557082983833669
precisions          : [0.2016341785138502, 0.12024287927841674]
brevity_penalty     : 1.0
length_ratio        : 2.2177799447424023
translation_length  : 81876
reference_length    : 36918
******************************************************************************************
                                     B

## Evaluate on the Test set

In [9]:
with open(f"{config.VAL_DATA}/english.txt") as data:
    english_val = data.read().strip().split("\n")
with open(f"{config.VAL_DATA}/afrikaans.txt") as data:
    afrikaans_val = data.read().strip().split("\n")

### Greedy Search

In [10]:
VAL_AF_REF = [[sent] for sent in afrikaans_val]
VAL_TRANSLATED = [translator.translate_sentence(sent) for sent in english_val]

corpus_bleu(VAL_TRANSLATED, VAL_AF_REF)

                                     BLEU-1                                     
------------------------------------------------------------------------------------------
bleu                : 0.19622017070196832
precisions          : [0.1962201707019683]
brevity_penalty     : 1.0
length_ratio        : 2.3589919189152173
translation_length  : 34446
reference_length    : 14602
******************************************************************************************
                                     BLEU-2                                     
------------------------------------------------------------------------------------------
bleu                : 0.1521855211878542
precisions          : [0.1962201707019683, 0.1180328850819132]
brevity_penalty     : 1.0
length_ratio        : 2.3589919189152173
translation_length  : 34446
reference_length    : 14602
******************************************************************************************
                                     BL

### Beam search

In [11]:
VAL_TRANSLATED = [translator.translate_sentence(sent, method="beam", beam_width=3) for sent in english_val]
corpus_bleu(VAL_TRANSLATED, VAL_AF_REF)

                                     BLEU-1                                     
------------------------------------------------------------------------------------------
bleu                : 0.13085785182559373
precisions          : [0.13085785182559376]
brevity_penalty     : 1.0
length_ratio        : 3.8638542665388305
translation_length  : 56420
reference_length    : 14602
******************************************************************************************
                                     BLEU-2                                     
------------------------------------------------------------------------------------------
bleu                : 0.09969493294427056
precisions          : [0.13085785182559376, 0.07595325397828875]
brevity_penalty     : 1.0
length_ratio        : 3.8638542665388305
translation_length  : 56420
reference_length    : 14602
******************************************************************************************
                                   

## Evaluate on the SUN validation set only

In [12]:
with open(f"{config.VAL_DATA}/sun_english.txt") as data:
    sun_english_val = data.read().strip().split("\n")
with open(f"{config.VAL_DATA}/sun_afrikaans.txt") as data:
    sun_afrikaans_val = data.read().strip().split("\n")

### Greedy Search

In [13]:
SUN_VAL_AF = [[sent] for sent in sun_afrikaans_val]
SUN_VAL_TRANSLATED = [translator.translate_sentence(sent) for sent in sun_english_val]
corpus_bleu(SUN_VAL_TRANSLATED, SUN_VAL_AF)

                                     BLEU-1                                     
------------------------------------------------------------------------------------------
bleu                : 0.25943396226415094
precisions          : [0.25943396226415094]
brevity_penalty     : 1.0
length_ratio        : 1.556371263765076
translation_length  : 5936
reference_length    : 3814
******************************************************************************************
                                     BLEU-2                                     
------------------------------------------------------------------------------------------
bleu                : 0.18278375269664202
precisions          : [0.25943396226415094, 0.12877997914494266]
brevity_penalty     : 1.0
length_ratio        : 1.556371263765076
translation_length  : 5936
reference_length    : 3814
******************************************************************************************
                                     BLEU

### Beam Search

In [14]:
SUN_VAL_TRANSLATED = [translator.translate_sentence(sent, method="beam", beam_width=3) for sent in sun_english_val]
corpus_bleu(SUN_VAL_TRANSLATED, SUN_VAL_AF)

                                     BLEU-1                                     
------------------------------------------------------------------------------------------
bleu                : 0.1399737876802097
precisions          : [0.1399737876802097]
brevity_penalty     : 1.0
length_ratio        : 3.000786575773466
translation_length  : 11445
reference_length    : 3814
******************************************************************************************
                                     BLEU-2                                     
------------------------------------------------------------------------------------------
bleu                : 0.09596335417303216
precisions          : [0.1399737876802097, 0.0657906419248868]
brevity_penalty     : 1.0
length_ratio        : 3.000786575773466
translation_length  : 11445
reference_length    : 3814
******************************************************************************************
                                     BLEU-3

In [15]:
metric = evaluate.load("bleu")
predictions = [translator.translate_sentence(sent, method="beam", beam_width=5) for sent in sun_english_val[10:20]]
labels = SUN_VAL_AF[10:20]
for source, pred, lab in zip(sun_english_val[10:20],predictions, labels):
    print(f"Source    : {source}")
    print(f"Prediction: {pred[:150]}")
    print(f"Label     : {lab[0][:150]}")
    print(f"BLEU      : {metric.compute(predictions=[pred], references=lab)['bleu']}")
    print()

Source    : <sos> component <eos>
Prediction: <sos> die sein $x(t)$ word nou teen 'n monsterfrekwensie van $f_s=2000\textrm{hz}$ gemonster <eos>
Label     : <sos> komponent <eos>
BLEU      : 0.0

Source    : <sos> architecture <eos>
Prediction: <sos> die sein $x(t)$ word nou teen 'n monsterfrekwensie van $f_s=2000\textrm{hz}$ gemonster <eos>
Label     : <sos> argitektuur <eos>
BLEU      : 0.0

Source    : <sos> specification <eos>
Prediction: <sos> die sein $x(t)$ word nou teen 'n monsterfrekwensie van $f_s=2000\textrm{hz}$ gemonster <eos>
Label     : <sos> spesifikasies <eos>
BLEU      : 0.0

Source    : <sos> at which stage of the design process would we choose the communication protocol between subsystems <eos>
Prediction: <sos> veronderstel ons het 'n datastel met vyf kenmerke , $x_1$ = matriek gemiddeld , $x_2$ = ik toetspunt , $x_3$ = geslag (1 vir vroulik en 0 vir ma
Label     : <sos> by watter stap van die ontwerpsproses word die kommunikasie-kanaal tussen substelsels gekies <e